## Code for importing (manual) QC files from various raters
### Gleb: MNI, JB 
- 1: good image 
- +-1: acceptable image 
- -1: unacceptable image 
- 0 - really bad image

### maarten: UMass, DK
- Comprises multiple stages of QC
- Complete Pass: '0s' in all non-age QC columns

In [ ]:
# basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read Data

In [35]:
proj_dir = '/home/nikhil/projects/CT_reproduce/'
data_dir = proj_dir + 'data/'
qc_dir = data_dir + 'QC/'
gleb_abide1 = qc_dir + 'John_Gleb/QCfile_20161104_141502_noTypos.csv' 
maarten_abide1 = qc_dir + 'DK/fenotype_data.csv' 

In [44]:
gleb_abide1_df = pd.read_csv(gleb_abide1,header=None,delimiter=',')
maarten_abide1_df = pd.read_csv(maarten_abide1,dtype={'SUB_ID': str})
gleb_abide1_df = gleb_abide1_df.rename(columns={0:'subject_id_long',1:'QC_description'})

### Parse Gleb data

In [45]:
tmp_df = gleb_abide1_df['subject_id_long'].str.split('+', 2, expand=True)
gleb_abide1_df['QC_gleb'], gleb_abide1_df['description_gleb'] = gleb_abide1_df['QC_description'].str.split('(', 1).str

gleb_abide1_df[['Site_ID','SubjID']]=tmp_df[[0,1]]
gleb_abide1_df = gleb_abide1_df[['Site_ID','SubjID','QC_gleb','description_gleb']]
gleb_abide1_df['QC_gleb'] = gleb_abide1_df['QC_gleb'].str.strip()
gleb_abide1_df.tail()

,Site_ID,SubjID,QC_gleb,description_gleb
1026,Yale,0050624,-1,NaN
1027,Yale,0050625,-1,NO IMAGES AVAILABLE FOR THIS SCAN!)
1028,Yale,0050626,-1,NaN
1029,Yale,0050627,-1,NaN
1030,Yale,0050628,-1,NaN


### Parse Maarten data

In [46]:
maarten_abide1_df[['filter_FIQ','filter_MRI','filter_missingMRI','filter_manualMRI']].astype('int')
maarten_abide1_df['QC_maarten'] = maarten_abide1_df.loc[:,['filter_FIQ','filter_MRI','filter_missingMRI','filter_manualMRI']].sum(axis=1)
maarten_abide1_df['SubjID']='00' + maarten_abide1_df['SUB_ID']
maarten_abide1_df.head()

,SITE_ID,SUB_ID,filter_age,filter_FIQ,filter_MRI,filter_missingMRI,filter_manualMRI,QC_maarten,SubjID
0,1,51456,1,0,0,0,0,0,0051456
1,1,51457,0,0,0,0,0,0,0051457
2,1,51458,1,0,0,0,0,0,0051458
3,1,51459,0,0,0,0,0,0,0051459
4,1,51460,1,1,0,0,0,1,0051460


### Create merged master

In [41]:
master_QC_df = pd.merge(gleb_abide1_df,maarten_abide1_df[['SubjID','QC_maarten']],on='SubjID',how='inner')
master_QC_df.head()

,Site_ID,SubjID,QC_gleb,description_gleb,QC_maarten
0,Caltech,0051456,-1,gradient too high around sensorimotor strip an...,0
1,Caltech,0051457,-1,gradient too high around sm strip and to some ...,0
2,Caltech,0051458,-1,slightly too high gradient on sm strip,0
3,Caltech,0051459,-1,more problematic in V1 than in sm strip; some ...,0
4,Caltech,0051460,-1,V1 worse than sm strip),1


In [43]:
#master_QC_df.to_csv(qc_dir + 'master_QC_table.csv')